# Model for single mode squeezed vacuum and squeezed coherent state

We test the boson sampling operator (no training) 
on coherent states and squeezed states, and 
propagating in a random medium

nonlinearxwaves@gmail.com<br>
@version 28 January 2021<br>
@version 1 October 2023 <br>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warning messages 

In [2]:
import numpy as np
from thqml import phasespace as ps
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
tf_complex = tf.complex
tf_real = tf.float32
np_complex = complex
np_real = np.float64

In [4]:
tf.keras.backend.clear_session()

In [5]:
np.set_printoptions(precision=2)

Check GPU

In [6]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


## Dimension

In [7]:
N = 4

In [8]:
n=np.floor_divide(N,2)

## Build vacuum by the Gaussian state

In [9]:
vacuum = ps.VacuumLayer(N)

## Squeezer for mode 0

In [10]:
r_np=0.8;
theta_np=np.pi/4;
squeezer=ps.SingleModeSqueezerLayer(N, r_np=r_np, theta_np=theta_np, n_squeezed=0)

## Displacer for mode 1

Define the alpha vector for the alpha

In [11]:
alpha=np.zeros([2],dtype=np_complex)

In [12]:
alpha[0]=0
alpha[1]=1.5
print(alpha)
print(np.abs(alpha)**2)

[0. +0.j 1.5+0.j]
[0.   2.25]


Define the corresponding displacement vector

In [13]:
dinput_np=np.zeros([N,1], dtype=np_real)
for j in range(n):
    dinput_np[2*j]=np.sqrt(2)*np.real(alpha[j])
    dinput_np[2*j+1]=np.sqrt(2)*np.imag(alpha[j])

In [14]:
dinput = tf.constant( dinput_np, dtype=vacuum.dtype)
displacer = ps.DisplacementLayerConstant(dinput)

## Random medium (non trainable)

In [15]:
R=ps.RandomLayerConstant(N)

## Build the model

In [16]:
xin = tf.keras.layers.Input(N)
x1, a1 = R(xin)
x2, a2 = displacer(x1, a1)
x3, a3 = squeezer(x2, a2)
chir, chii = vacuum(x3, a3)
model = tf.keras.Model(inputs = xin, outputs=[chir, chii])

# Evaluate the covariance

In [17]:
cov_layer = ps.CovarianceLayer(N)
covariance_matrix, mean_R, hessian = cov_layer(chir,chii, model)
squeezed_cov = tf.keras.Model(inputs = xin, outputs=[covariance_matrix, mean_R, hessian])
xtrain = np.random.rand(1, N)-0.5
cov0,d0, hessian0=squeezed_cov(xtrain); print(d0); tf.print(cov0)

tf.Tensor([[-1.63  1.27  0.47  0.14]], shape=(1, 4), dtype=float32)
[[1.19323635 0.0602378845 0.271396399 0.813356757]
 [0.0602374077 0.973291159 0.240955591 0.14092207]
 [0.271396279 0.240955591 0.843739033 1.52946949]
 [0.813356638 0.14092207 1.52946925 4.14466143]]


# Evaluate the Qtransform with calculated covariance and displacemente

In [18]:
Qrho = ps.QTransform(cov0, d0)

# Test the Pr operator 

Probability of zero photon

In [19]:
%%time
ps.Pr([0,0], Qrho)

CPU times: user 13.9 ms, sys: 0 ns, total: 13.9 ms
Wall time: 12.7 ms


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.08]], dtype=float32)>

Probability of two photons

In [20]:
%%time
ps.Pr([0,2], Qrho)

CPU times: user 102 ms, sys: 3.27 ms, total: 105 ms
Wall time: 104 ms


<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.01], dtype=float32)>

## Some numerical examples

In [21]:
%%time
nbar=[0,0]
tf.print(ps.Pr(nbar, Qrho))

[[0.078807056]]
CPU times: user 6.01 ms, sys: 0 ns, total: 6.01 ms
Wall time: 5.12 ms


In [22]:
%%time
nbar=[1,0]
tf.print(ps.Pr(nbar, Qrho))

[0.167956561]
CPU times: user 21.6 ms, sys: 189 µs, total: 21.8 ms
Wall time: 20.8 ms


In [23]:
%%time
nbar=[1,1]
tf.print(ps.Pr(nbar, Qrho))

[0.02463804]
CPU times: user 101 ms, sys: 3.13 ms, total: 104 ms
Wall time: 103 ms


In [24]:
%%time
nbar=[2,3]
tf.print(ps.Pr(nbar, Qrho))

[0.0171649]
CPU times: user 27.9 s, sys: 72.7 ms, total: 28 s
Wall time: 28 s


## Make a plot versus the probability

Build an array of photon number sampling

In [25]:
%%time
nmax=3 # max number of photons
# generate the combinations
import itertools as it
nlist=it.product(range(nmax+1),repeat=n)
ln=list(nlist)
print(ln)
print(len(ln))

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]
16
CPU times: user 0 ns, sys: 221 µs, total: 221 µs
Wall time: 211 µs


In [ ]:
%%time
npatterns=len(ln)
Pn = np.zeros([npatterns,], dtype=np.float32)
Pth = np.zeros_like(Pn)
xaxis=np.zeros_like(Pn)
for nbar in range(npatterns):
    print('Computing Pn at n '+repr(ln[nbar]))
    Pn[nbar]=ps.Pr(ln[nbar],Qrho).numpy()
    tf.print('Computed by the neural network: '+repr(Pn[nbar]))
    xaxis[nbar]=nbar #store for plot
    print('--------------------')
print('Done')

Computing Pn at n (0, 0)
Computed by the neural network: 0.078807056
--------------------
Computing Pn at n (0, 1)
Computed by the neural network: 0.009359261
--------------------
Computing Pn at n (0, 2)
Computed by the neural network: 0.014667837
--------------------
Computing Pn at n (0, 3)
Computed by the neural network: 0.0054007284
--------------------
Computing Pn at n (1, 0)
Computed by the neural network: 0.16795656
--------------------
Computing Pn at n (1, 1)
Computed by the neural network: 0.02463804
--------------------
Computing Pn at n (1, 2)
Computed by the neural network: 0.024618601
--------------------
Computing Pn at n (1, 3)
Computed by the neural network: 0.013319234
--------------------
Computing Pn at n (2, 0)
Computed by the neural network: 0.17754915
--------------------
Computing Pn at n (2, 1)
Computed by the neural network: 0.03437425
--------------------
Computing Pn at n (2, 2)
Computed by the neural network: 0.019022206
--------------------
Computing Pn 

In [ ]:
plt.bar(xaxis,Pn)
plt.xlabel(ln, fontsize=6);
plt.ylabel('Sample Probability Pr');
plt.savefig('BosonSamplingExample5.pdf', format='pdf', bbox_inches='tight')

Probability of the photon patterns
Sampling patters are indicate in the labels.